In [1]:
# coding: utf-8
"""
SubModularを実験するためのコード
"""
from filer2.filer2 import Filer
from library.submodular import OpinionSubModular
from library.rouge import Rouge_N
import numpy as np
import glob

In [2]:
start = 0
end = 18

list_test_path = glob.glob('./files/rakuten_ichiba/dumpfile/test/*')
list_ans_path = glob.glob('./files/rakuten_ichiba/dumpfile/ans/*')

list_test_path.sort()
list_ans_path.sort()

list_test_path = list_test_path[start:end]
list_ans_path = list_ans_path[start:end]

print len(list_test_path)

18


In [ ]:
counter = 1
for test_path, ans_path in zip(list_test_path, list_ans_path):
    dict_test = Filer.readdump(test_path)
    dict_ans = Filer.readdump(ans_path)
    filename = ans_path.replace('./files/rakuten_ichiba/dumpfile/ans/', '')
    filename = filename.replace('.dump', '')
    print counter
    counter += 1
    # テスト用ファイルの作成
    # １００単語以上ある文は除く
    list_sentence = []
    list_sep = []
    list_sep_lemmas = []
    for row in dict_test:
        sepall = row['sep_all']
        lemmas = row['sep_all_lemmas']
        sen = row['sentence']
        if len(sepall) <= 100 and len(sepall) != 0:
            list_sentence.append(sen)
            list_sep.append(sepall)
            list_sep_lemmas.append(lemmas)
    
    # インスタンス化
    OSM = OpinionSubModular(list_sep=list_sep,
                            list_sep_lemmas=list_sep_lemmas,
                            dict_path='./files/dictionary/dict_word_pol_japanese.dump')
    for alpha in [0.5]:
        for lamda in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:
            for r in [0.1, 0.5, 1, 2]:
                # 学習
                n_cluster = len(list_sep)/5
                OSM.m_greedy(num_w=100,
                             alpha=alpha,
                             lamda=lamda,
                             r=r,
                             n_cluster=n_cluster)
                list_C = OSM.list_C
                # Rougeを計算するようにbag_of_wordsを作成する
                list_sm_test = [list_sep[row[0]] for row in list_C]
                list_sm_ans = dict_ans['sep_all']
                try:
                    R1 = Rouge_N.rouge(list_sm_test, list_sm_ans, N=1)
                except:
                    R1 = 0
                try:
                    R2 = Rouge_N.rouge(list_sm_test, list_sm_ans, N=2)
                except:
                    R2 = 0
                try:
                    R3 = Rouge_N.rouge(list_sm_test, list_sm_ans, N=3)
                except:
                    R3 = 0
                try:
                    R4 = Rouge_N.rouge(list_sm_test, list_sm_ans, N=4)
                except:
                    R4 = 0
                list_result = [[filename, alpha, lamda, r, R1, R2, R3, R4]]
                Filer.writecsv(list_result, './files/rakuten_ichiba/result/OSM/all_%s_%s_%s.csv'%(alpha, lamda, r))

1
num_sentence:  102
num word:  1861
num vocabulary:  399
計算が終了しました
property: list_C
計算が終了しました
property: list_C
計算が終了しました